# Texas Tow Trucks

We're going to scrape some [tow trucks in Texas](https://www.tdlr.texas.gov/tools_search/).

## Import your imports

In [74]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait

## Search for the TLDR Number `006179570C`, and scrape the information on that company

Using [license information system](https://www.tdlr.texas.gov/tools_search/), find information about the tow truck number above, displaying the

- The business name
- Owner/operator
- Phone number
- License status (Active, Expired, Etc)
- Physical address

If you can't figure a 'nice' way to locate something, your two last options might be:

- **Find a "parent" element, then dig inside**
- **Find all of a type of element** (like we did with `td` before) and get the `[0]`, `[1]`, `[2]`, etc
- **XPath** (inspect an element, Copy > Copy XPath)

These kinds of techniques tend to break when you're on other result pages, but... maybe not! You won't know until you try.

> - *TIP: When you use xpath, you CANNOT use double quotes or Python will get confused. Use single quotes.*
> - *TIP: You can clean your data up if you want to, or leave it dirty to clean later*
> - *TIP: The address part can be tough, but you have a few options. You can use a combination of `.split` and list slicing to clean it now, or clean it later in the dataframe with regular expressions. Or other options, too, probably*

In [4]:
driver = webdriver.Chrome()
driver.get('https://www.tdlr.texas.gov/tools_search/')
search = driver.find_element_by_id('mcrdata')
search.send_keys("006179570C")

In [5]:
button = driver.find_element_by_id('submit3')
button.click()

In [33]:
name = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[2]/tbody/tr[2]/td[1]').text
name_stripped = name.replace("Name:", "")
#driver.find_elements_by_tag_name('td')[5].text
owner = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[2]/tbody/tr[3]/td[1]').text
owner_stripped = owner.replace("Owner/Officer:", "") 
phone = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[2]/tbody/tr[4]/td[1]').text
phone_stripped = phone.replace("Phone:", "")
status = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[3]/tbody/tr[1]/td[2]/font').text
addressinfo = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[3]/tbody/tr[2]/td[2]').text
address = addressinfo.split("\n")[-2:]

# Adapt this to work inside of a single cell

Double-check that it works. You want it to print out all of the details.

In [34]:
driver = webdriver.Chrome()
driver.get('https://www.tdlr.texas.gov/tools_search/')
search = driver.find_element_by_id('mcrdata')
search.send_keys("006179570C")
button = driver.find_element_by_id('submit3')
button.click()

name = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[2]/tbody/tr[2]/td[1]').text
name_stripped = name.replace("Name:", "")
owner = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[2]/tbody/tr[3]/td[1]').text
owner_stripped = owner.replace("Owner/Officer:", "") 
phone = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[2]/tbody/tr[4]/td[1]').text
phone_stripped = phone.replace("Phone:", "")
status = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[3]/tbody/tr[1]/td[2]/font').text
addressinfo = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[3]/tbody/tr[2]/td[2]').text
address = addressinfo.split("\n")[-2:]

print("Name is", name_stripped)
print("Owner is", owner_stripped)
print("Phone number is", phone_stripped)
print("Status is", status)
print("Physical address is", address)

Name is    B.D. SMITH TOWING
Owner is    BRANDT SMITH / OWNER
Phone number is    8173330706
Status is Expired
Physical address is ['13619 BRETT JACKSON RD.', 'FORT WORTH, TX. 76179']


# Using .apply to find data about SEVERAL tow truck companies

The file `trucks-subset.csv` has information about the trucks, we'll use it to find the pages to scrape.

### Open up `trucks-subset.csv` and save it into a dataframe

In [37]:
import pandas as pd
df = pd.read_csv('trucks-subset.csv')
df

,TDLR Number
0,006507931C
1,006179570C
2,006502097C


## Go through each row of the dataset, displaying the URL you will need to scrape for the information on that row

You don't have to actually use the search form for each of these - look at the URL you're on, it has the number in it!

For example, one URL might look like `https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006495492C`.

- *TIP: Use .apply and a function*
- *TIP: Unlike the Yelp example, you'll need to build this URL from pieces*
- *TIP: You probably don't want to `print` unless you're going to fix it for the next question 
- *TIP: pandas won't showing you the entire url! Run `pd.set_option('display.max_colwidth', -1)` to display aaaalll of the text in a cell*

In [48]:
def get_tdlr_url(row):
    url_pattern = "https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber={}"
    url = url_pattern.format(row['TDLR Number'])
    return url

### Save this URL into a new column of your dataframe, called `url`

- *TIP: Use a function and `.apply`*
- *TIP: Be sure to use `return`*

In [49]:
pd.set_option('display.max_colwidth', -1)
df['url'] = df.apply(get_tdlr_url, axis=1)
df.head()

,TDLR Number,url
0,006507931C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006507931C
1,006179570C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006179570C
2,006502097C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006502097C


## Go through each row of the dataset, printing out information about each tow truck company.

Now will be **scraping** inside of your function.

- The business name
- Owner/operator
- Phone number
- License status (Active, Expired, Etc)
- Physical address

Just print it out for now.

- *TIP: use .apply*
- *TIP: You'll be using the code you wrote before, but converted into a function*
- *TIP: Remember how the TDLR Number is in the URL? You don't need to do the form submission if you don't want!*
- *TIP: Make sure you adjust any variables so you don't scrape the same page again and again*

In [61]:
def get_subset_info(row):
    #Visit the URL from the row
    driver.get(row['url'])
    
    
    name = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[2]/tbody/tr[2]/td[1]').text
    name_stripped = name.replace("Name:", "")
    owner = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[2]/tbody/tr[3]/td[1]').text
    owner_stripped = owner.replace("Owner/Officer:", "") 
    phone = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[2]/tbody/tr[4]/td[1]').text
    phone_stripped = phone.replace("Phone:", "")
    status = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[3]/tbody/tr[1]/td[2]/font').text
    addressinfo = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[3]/tbody/tr[2]/td[2]').text
    address = addressinfo.split("\n")[-2:]

    print("Name is", name_stripped)
    print("Owner is", owner_stripped)
    print("Phone number is", phone_stripped)
    print("Status is", status)
    print("Physical address is", address)
    

In [63]:
df.apply(get_subset_info, axis=1)

Name is    AUGUSTUS E SMITH
Owner is    AUGUSTUS EUGENE SMITH / OWNER
Phone number is    9032276464
Status is Active
Physical address is ['103 N MAIN ST', 'BONHAM, TX. 75418']
Name is    B.D. SMITH TOWING
Owner is    BRANDT SMITH / OWNER
Phone number is    8173330706
Status is Expired
Physical address is ['13619 BRETT JACKSON RD.', 'FORT WORTH, TX. 76179']
Name is    BARRY MICHAEL SMITH
Owner is    BARRY MICHAEL SMITH / OWNER
Phone number is    8066544404
Status is Active
Physical address is ['4501 W CEMETERY RD', 'CANYON, TX. 79015']


0    None
1    None
2    None
dtype: object

## Scrape the following information for each row of the dataset, and save it into new columns in your dataframe.

- The business name
- Owner/operator
- Phone number
- License status (Active, Expired, Etc)
- Physical address

It's basically what we did before, but using the function a little differently.

- *TIP: Same as above, but you'll be returning a `pd.Series` and the `.apply` line is going to be a lot longer*
- *TIP: Save it to a new dataframe!*
- *TIP: Make sure you change your `df` variable names correctly if you're cutting and pasting - there are a few so it can get tricky*

In [66]:
def get_subset_info(row):
    #Visit the URL from the row
    driver.get(row['url'])
    
    
    name = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[2]/tbody/tr[2]/td[1]').text
    name_stripped = name.replace("Name:", "")
    owner = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[2]/tbody/tr[3]/td[1]').text
    owner_stripped = owner.replace("Owner/Officer:", "") 
    phone = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[2]/tbody/tr[4]/td[1]').text
    phone_stripped = phone.replace("Phone:", "")
    status = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[3]/tbody/tr[1]/td[2]/font').text
    addressinfo = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[3]/tbody/tr[2]/td[2]').text
    address = addressinfo.split("\n")[-2:]

    return pd.Series({
        'name': name_stripped,
        'owner/operator': owner_stripped,
        'phone number': phone_stripped,
        'license status': status,
        'physical address': address
    })

In [67]:
towtrucks_subset_df = df.apply(get_subset_info, axis=1).join(df)
towtrucks_subset_df

,name,owner/operator,phone number,license status,physical address,TDLR Number,url
0,AUGUSTUS E SMITH,AUGUSTUS EUGENE SMITH / OWNER,9032276464,Active,"[103 N MAIN ST, BONHAM, TX. 75418]",006507931C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006507931C
1,B.D. SMITH TOWING,BRANDT SMITH / OWNER,8173330706,Expired,"[13619 BRETT JACKSON RD., FORT WORTH, TX. 76179]",006179570C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006179570C
2,BARRY MICHAEL SMITH,BARRY MICHAEL SMITH / OWNER,8066544404,Active,"[4501 W CEMETERY RD, CANYON, TX. 79015]",006502097C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006502097C


### Save your dataframe as a CSV named `tow-trucks-extended.csv`

In [68]:
towtrucks_subset_df.to_csv("tow-trucks-extended.csv", index=False)

### Re-open your dataframe to confirm you didn't save any extra weird columns

In [72]:
pd.read_csv('tow-trucks-extended.csv')

,name,owner/operator,phone number,license status,physical address,TDLR Number,url
0,AUGUSTUS E SMITH,AUGUSTUS EUGENE SMITH / OWNER,9032276464,Active,"['103 N MAIN ST', 'BONHAM, TX. 75418']",006507931C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006507931C
1,B.D. SMITH TOWING,BRANDT SMITH / OWNER,8173330706,Expired,"['13619 BRETT JACKSON RD.', 'FORT WORTH, TX. 76179']",006179570C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006179570C
2,BARRY MICHAEL SMITH,BARRY MICHAEL SMITH / OWNER,8066544404,Active,"['4501 W CEMETERY RD', 'CANYON, TX. 79015']",006502097C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006502097C


## Process the entire `tow-trucks.csv` file

We just did it on a short subset so far. Now try it on all of the tow trucks. **Save as the same filename as before**

In [77]:
df = pd.read_csv('tow-trucks.csv')
df.head()

,TDLR Number
0,006507931C
1,006179570C
2,006502097C
3,006494912C
4,0649468VSF


In [79]:
pd.set_option('display.max_colwidth', -1)
df['url'] = df.apply(get_tdlr_url, axis=1)
df

,TDLR Number,url
0,006507931C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006507931C
1,006179570C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006179570C
2,006502097C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006502097C
3,006494912C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006494912C
4,0649468VSF,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=0649468VSF
5,006448786C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006448786C
6,0648444VSF,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=0648444VSF
7,0651667VSF,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=0651667VSF
8,006017767C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006017767C
9,006495492C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006495492C


In [80]:
def get_subset_info(row):
    #Visit the URL from the row
    driver.get(row['url'])
    
    
    name = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[2]/tbody/tr[2]/td[1]').text
    name_stripped = name.replace("Name:", "")
    owner = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[2]/tbody/tr[3]/td[1]').text
    owner_stripped = owner.replace("Owner/Officer:", "") 
    phone = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[2]/tbody/tr[4]/td[1]').text
    phone_stripped = phone.replace("Phone:", "")
    status = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[3]/tbody/tr[1]/td[2]/font').text
    addressinfo = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[3]/tbody/tr[2]/td[2]').text
    address = addressinfo.split("\n")[-2:]

    return pd.Series({
        'name': name_stripped,
        'owner/operator': owner_stripped,
        'phone number': phone_stripped,
        'license status': status,
        'physical address': address
    })

In [84]:
towtrucks_df = df.apply(get_subset_info, axis=1).join(df)
towtrucks_df.head()

,name,owner/operator,phone number,license status,physical address,TDLR Number,url
0,AUGUSTUS E SMITH,AUGUSTUS EUGENE SMITH / OWNER,9032276464,Active,"[103 N MAIN ST, BONHAM, TX. 75418]",006507931C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006507931C
1,B.D. SMITH TOWING,BRANDT SMITH / OWNER,8173330706,Expired,"[13619 BRETT JACKSON RD., FORT WORTH, TX. 76179]",006179570C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006179570C
2,BARRY MICHAEL SMITH,BARRY MICHAEL SMITH / OWNER,8066544404,Active,"[4501 W CEMETERY RD, CANYON, TX. 79015]",006502097C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006502097C
3,HEATH SMITH,HEATH A SMITH / OWNER,940-552-0687,Expired,"[1529 WILBARGER ST, VERNON, TX. 76384]",006494912C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006494912C
4,HEATH SMITH,HEATH A SMITH / OWNER,9405520687,Expired,"[1529 WILBARGER ST, VERNON, TX. 76384]",0649468VSF,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=0649468VSF


In [82]:
towtrucks_df.to_csv("tow-trucks.csv", index=False)

In [83]:
#confirming that it saved correctly
pd.read_csv('tow-trucks.csv')

,name,owner/operator,phone number,license status,physical address,TDLR Number,url
0,AUGUSTUS E SMITH,AUGUSTUS EUGENE SMITH / OWNER,9032276464,Active,"['103 N MAIN ST', 'BONHAM, TX. 75418']",006507931C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006507931C
1,B.D. SMITH TOWING,BRANDT SMITH / OWNER,8173330706,Expired,"['13619 BRETT JACKSON RD.', 'FORT WORTH, TX. 76179']",006179570C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006179570C
2,BARRY MICHAEL SMITH,BARRY MICHAEL SMITH / OWNER,8066544404,Active,"['4501 W CEMETERY RD', 'CANYON, TX. 79015']",006502097C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006502097C
3,HEATH SMITH,HEATH A SMITH / OWNER,940-552-0687,Expired,"['1529 WILBARGER ST', 'VERNON, TX. 76384']",006494912C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006494912C
4,HEATH SMITH,HEATH A SMITH / OWNER,9405520687,Expired,"['1529 WILBARGER ST', 'VERNON, TX. 76384']",0649468VSF,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=0649468VSF
5,HYSMITH AUTOMOTIVE,WILLIAM THOMAS HYSMITH / PRESIDENT,Owner/Officer: ASHLEY ERIN HYSMITH / TREASURER,Active,"['1210 US 380 BYPASS', 'GRAHAM, TX. 76450']",006448786C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006448786C
6,HYSMITH AUTOMOTIVE & TRUCK REPAIR INC,HYSMITH ERIN ASHLEY / TREASURER,Owner/Officer: WILLIAM THOMAS HYSMITH / PRESIDENT,Suspended,"['927 LOVING HWY', 'GRAHAM, TX. 76450']",0648444VSF,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=0648444VSF
7,HYSMITH AUTOMOTIVE & TRUCK REPAIR INC,WILLIAM THOMAS HYSMITH / PRESIDENT,Owner/Officer: ASHLEY ERIN HYSMITH / TREASURER,Active,"['1210 380 BYPASS', 'GRAHAM, TX. 76450']",0651667VSF,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=0651667VSF
8,JEFF & WENDY SMITH,JEFF SMITH / PARTNER,Owner/Officer: WENDY SMITH / PARTNER,Suspended,"['10842 FM 2138 N', 'JACKSONVILLE, TX. 75766']",006017767C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006017767C
9,JEFF SMITH,JEFFREY JOHN SMITH / OWNER,8324354670,Active,"['4338 HARVEY RD', 'CROSBY, TX. 77532']",006495492C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006495492C
